In [1]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf

print ( tf.__version__)

1.3.0


In [13]:
df=pd.read_csv('conversion_data.csv')

In [14]:
df.head(3)

,country,age,new_user,source,total_pages_visited,converted
0,UK,25,1,Ads,1,0
1,US,23,1,Seo,5,0
2,US,28,1,Seo,4,0


In [15]:
df.country.value_counts()

US         178092
China       76602
UK          48450
Germany     13056
Name: country, dtype: int64

In [16]:
df.source.value_counts()

Seo       155040
Ads        88740
Direct     72420
Name: source, dtype: int64

In [17]:
# sampling 70% for train data
train_set = df.sample(frac=0.7, replace=False, random_state=777)
# the other 30% is reserverd for testing
test_set = df.loc[ set(df.index) - set(train_set.index)]

In [18]:
train_set.shape

(221340, 6)

In [19]:
test_set.shape

(94860, 6)

In [20]:
# defining numeric columns
age_feature = tf.feature_column.numeric_column('age')
new_user_feature = tf.feature_column.numeric_column('new_user')
total_pages_visited_feature = tf.feature_column.numeric_column('total_pages_visited')


In [21]:
#defining a categorical column with predefined values
country_feature = tf.feature_column.categorical_column_with_vocabulary_list(
    'country',['US','UK','China','Germany']
)

source_feature = tf.feature_column.categorical_column_with_vocabulary_list(
    'source',['Ads','Seo','Direct']
)


In [22]:
feature_columns = [ age_feature,new_user_feature, total_pages_visited_feature, country_feature, 
                   source_feature]

In [23]:
estimator = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgrz9upmr', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [25]:
# train input function
train_input_fn = tf.estimator.inputs.pandas_input_fn(
      x=train_set.drop('converted', axis=1),
      y=train_set.converted,
      num_epochs=None, #For training it can use how many epochs is necessary
      shuffle=True,
      target_column='target',
)

test_input_fn = tf.estimator.inputs.pandas_input_fn(
      x=test_set.drop('converted', axis=1),
      y=test_set.converted,
      num_epochs=1, #We just want to use one epoch since this is only to score.
      shuffle=False  #It isn't necessary to shuffle the cross validation 
)

In [26]:
estimator.train(input_fn=train_input_fn, steps=400)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpgrz9upmr/model.ckpt.
INFO:tensorflow:loss = 88.7229, step = 1
INFO:tensorflow:global_step/sec: 346.507
INFO:tensorflow:loss = 3.24922, step = 101 (0.292 sec)
INFO:tensorflow:global_step/sec: 362.691
INFO:tensorflow:loss = 7.51745, step = 201 (0.273 sec)
INFO:tensorflow:global_step/sec: 406.578
INFO:tensorflow:loss = 11.9459, step = 301 (0.248 sec)
INFO:tensorflow:Saving checkpoints for 400 into /tmp/tmpgrz9upmr/model.ckpt.
INFO:tensorflow:Loss for final step: 5.72372.


In [27]:
scores = estimator.evaluate(input_fn=test_input_fn)
print("\nTest Accuracy: {0:f}\n".format(scores['accuracy']))

INFO:tensorflow:Starting evaluation at 2018-05-02-06:50:48
INFO:tensorflow:Restoring parameters from /tmp/tmpgrz9upmr/model.ckpt-400
INFO:tensorflow:Finished evaluation at 2018-05-02-06:50:51
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.983892, accuracy_baseline = 0.967816, auc = 0.974004, auc_precision_recall = 0.795024, average_loss = 0.0489862, global_step = 400, label/mean = 0.0321843, loss = 6.26258, prediction/mean = 0.0340002

Test Accuracy: 0.983892

